# 🧪 Pre‑Parcial – Métodos Predictivos
**Supervisado vs. No Supervisado**

> Completa las celdas con `TODO` y ejecuta *Runtime → Run all* antes de entregar.


### Contenido
1. [Parte A – Teoría](#parte-a)
2. [Parte B – Práctica](#parte-b)


## Parte A — Cuestionario Teórico (40 pts)
Responde **brevemente** en las celdas Markdown que siguen a cada pregunta.


#### 1️⃣ **Variable objetivo (y)** – Defínela y da un ejemplo en este dataset.

*Respuesta:* <!-- TODO -->

La variable objetivo (y) es la que se quiere predecir en un modelo. En un dataset de clientes, por ejemplo, la variable objetivo puede ser si realizan o no una compra.




#### 2️⃣ Ordena las fases del *pipeline* de ML: `Modelado`, `Pre‑procesamiento`, `EDA`, `Evaluación`, `Insight de negocio`.

*Respuesta:* <!-- TODO -->

Problema → EDA → Pre-procesamiento → Modelado → Evaluación → Insight de negocio
Este orden permite primero entender el problema desde el negocio, explorar los datos, prepararlos, construir el modelo, evaluarlo y tomar desiciones.




#### 3️⃣ Para un problema de **clases desbalanceadas**, ¿qué métrica priorizarías y por qué?

*Respuesta:* <!-- TODO -->

Priorizaría el F1-score porque equilibra precisión y recall, lo cual es vital cuando las clases no están distribuidas equitativamente. La exactitud (accuracy) puede ser engañosa si la clase minoritaria es la más importante.

#### 4️⃣ Describe **overfitting** y cómo lo detectarías en la práctica.

*Respuesta:* <!-- TODO -->

El overfitting es cuando un modelo aprende tanto los patrones como el ruido del conjunto de entrenamiento, lo que lo hace ineficiente al generalizar. Se detecta comparando el rendimiento, si el modelo tiene alta precisión en entrenamiento pero bajo desempeño en validación o test, probablemente hay sobreajuste.



#### 5️⃣ Completa: *K‑means es un algoritmo de _________ porque ________.*

*Respuesta:* <!-- TODO -->

K-means es un algoritmo de clustering no supervisado porque agrupa los datos en k clústeres sin requerir etiquetas, basándose en la similitud.



#### 7️⃣ En **regresión**, ¿cómo es la variable objetivo? (cualitativa, cuantitativa, binaria…).

*Respuesta:* <!-- TODO -->

Es cuantitativa ya que la regresión busca predecir valores numéricos como el precio de una casa o el ingreso mensual.


#### 8️⃣ Menciona 2 técnicas comunes de **pre‑procesamiento de texto**.

*Respuesta:* <!-- TODO -->

1. Normalización: Convertir el texto a un formato estándar, por ejemplo poner todo en minúsculas o eliminar signos de puntuación.

2. Eliminación de palabras vacías: Eliminar palabras comunes que no aportan mucho significado al análisis, por ejemplo "el" o "la".

#### 9️⃣ ¿Qué representa el parámetro *k* en K‑means y qué ocurre si es muy grande?

*Respuesta:* <!-- TODO -->

K representa el número de clústeres que queremos formar. Si se elige un valor muy grande, los clústeres pueden volverse demasiado pequeños, representando ruido o subdivisiones irrelevantes, lo cual puede perjudicar la interpretación y generalización del modelo.




#### 🔟 Define brevemente un **embedding** en NLP y su utilidad.

*Respuesta:* <!-- TODO -->

Un embedding es una representación numérica densa de palabras en un espacio vectorial. A diferencia del one-hot encoding, los embeddings capturan relaciones semánticas como por ejemplo "rey" y "reina" están cerca en el espacio. Son fundamentales en NLP porque permiten que los modelos comprendan mejor el significado contextual del lenguaje.

## <a id='parte-b'></a>💻 Parte B — Práctica (60 pts)
Trabajarás con la tabla de reseñas de Amazon que contiene, entre otras, las columnas `reviewerID`, `asin`, `helpful`, `reviewText`, `overall`, `summary`, `unixReviewTime`, `day_diff`.

> **Objetivos**
> 1. Clasificar reseñas positivas (rating ≥ 4) vs negativas.
> 2. Agrupar reseñas con K‑means y perfilar clusters.


Columnas de la tabla
📄 Reviewer ID: Unique identifier for the reviewer.

📦 ASIN: Amazon Standard Identification Number for the product.

👤 Reviewer Name: Name of the reviewer.

👍 Helpful: Number of helpful votes the review received.

📝 Review Text: The content of the review written by the customer.

⭐ Overall Rating: The overall rating given to the product (ranging from 1 to 5 stars).

📰 Summary: A brief summary of the review.

🕰️ Unix Review Time: The time the review was posted in Unix timestamp format.

📅 Review Time: The time the review was posted in a readable date format.

📆 Day Difference: The number of days between the review date and the current date.

✔️ Helpful Yes: Number of positive helpful votes.

🔢 Total Votes: Total number of votes the review received.

### 1. Setup

In [ ]:
# Instala paquetes extra si los necesitas
# !pip install shap

import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings, random, os
warnings.filterwarnings('ignore')
random.seed(42)
np.random.seed(42)

### 2. Carga del dataset

In [ ]:
!wget https://github.com/javierherrera1996/IntroMachineLearning/raw/refs/heads/main/TercerCorte/amazon_review.csv.zip

--2025-05-31 20:38:39--  https://github.com/javierherrera1996/IntroMachineLearning/raw/refs/heads/main/TercerCorte/amazon_review.csv.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMachineLearning/refs/heads/main/TercerCorte/amazon_review.csv.zip [following]
--2025-05-31 20:38:40--  https://raw.githubusercontent.com/javierherrera1996/IntroMachineLearning/refs/heads/main/TercerCorte/amazon_review.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 721801 (705K) [application/zip]
Saving to: ‘amazon_review.csv.zip.1’

amazon_review.csv.z 100%[===================>] 704

In [ ]:
!unzip amazon_review.csv.zip

Archive:  amazon_review.csv.zip
replace amazon_review.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
df = pd.read_csv('amazon_review.csv')
df.head()

### 3. Exploratory Data Analysis (EDA)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
print("\nDistribución de la columna 'overall' (rating):")
print(df['overall'].value_counts())
print(df['overall'].value_counts(normalize=True)) # En porcentajes
plt.figure(figsize=(8, 6))
sns.countplot(x='overall', data=df)
plt.title('Distribución de Ratings (overall)')
plt.xlabel('Rating')
plt.ylabel('Cantidad de Reseñas')
plt.show()

### 4. Limpieza & Feature Engineering

### 4.1 Cree un revie_datetime usando unixReviewTime y la funcion pd.to_datetime

In [ ]:
df['review_datetime'] = pd.to_datetime(df['unixReviewTime'], unit='s')
df.head()

### 4.2 Cree una columna positive que sea igual a 1 cuando overall sea mayor o igual  a 4.

In [ ]:
df['positive'] = (df['overall'] >= 4).astype(int)
df.head()

### 4.3 Cree una columna text que sea la union de summary y review text: text_cols['summary'] + '. ' + text_cols['reviewText']).str.strip()

In [ ]:
text_cols = df[['summary', 'reviewText']].fillna('') # fillna handles potential NaN values
df['text'] = (text_cols['summary'] + '. ' + text_cols['reviewText']).str.strip()
df.head()

### 4.4 Elimine las columnas que no puede usar en el modleo de clasificacion

In [ ]:
df = df.drop(['reviewerID', 'asin', 'reviewerName', 'helpful', 'overall', 'unixReviewTime', 'reviewTime', 'day_diff', 'helpful_yes', 'total_vote'], axis=1)
df.head()

In [ ]:
df.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
        df['text'], df['positive'], test_size=0.3, stratify=df['positive'], random_state=42)

pipe = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000)),
    ('clf',  LogisticRegression(max_iter=1000, class_weight='balanced', n_jobs=-1, random_state=42))
])

# pipe.fit(X_train, y_train)

### 5. Modelo de Clasificación – Supervisado (25 pts)
  * Ralice una regreison logistica y use la columa positive que creoo como target, entregue una matriz de confusion

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negativo (0)', 'Positivo (1)'], yticklabels=['Negativo (0)', 'Positivo (1)'])
plt.xlabel('Predicho')
plt.ylabel('Real')
plt.title('Matriz de Confusión')
plt.show()

In [ ]:
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))

### 6. Clustering K‑means – No Supervisado (20 pts)
Haga un modelo para agrupasr los comentarios: vectorizer = TfidfVectorizer(max_features=10000)
X_vec = vectorizer.fit_transform(df['text'])

y cre una variable que se llame clsuter

In [ ]:
vectorizer = TfidfVectorizer(max_features=10000)
X_vec = vectorizer.fit_transform(df['text'])

In [ ]:
k = 5
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10) # n_init para evitar advertencia
cluster = kmeans.fit_predict(X_vec)

In [ ]:
df['cluster'] = cluster
df.head()

In [ ]:
print("\nDistribución de reseñas por cluster:")
print(df['cluster'].value_counts())

#### Perfil de clusters: Entregue una descriptiva de que contenia cada clster en termino de las otras variables:

In [ ]:
# prompt: Perfil de clusters: Entregue una descriptiva de que contenia cada clster en termino de las otras variables

print("\n--- Perfil de Clusters ---")

# Iterar sobre cada cluster para analizar su perfil
for cluster_id in sorted(df['cluster'].unique()):
    print(f"\n--- Cluster {cluster_id} ---")

    # Filtrar el DataFrame para el cluster actual
    cluster_df = df[df['cluster'] == cluster_id]
    num_reviews = len(cluster_df)

    print(f"Total de reseñas en este cluster: {num_reviews}")

    # Análisis de la variable 'positive' (sentimiento binario)
    print("\nDistribución de Sentimiento (Positivo/Negativo):")
    print(cluster_df['positive'].value_counts(normalize=True)) # Mostrar porcentajes

    # Análisis de la variable 'helpful' (votos útiles) - estadísticas descriptivas
    # Asegúrate de que 'helpful' es numérico. Si es una cadena, puedes necesitar convertirla.
    # Aquí asumimos que 'helpful' en el df original ya era numérico o se limpió antes del drop.
    # Si usaste df_cleaned para el clustering, podrías necesitar re-unir las columnas relevantes.
    # Si el clustering se hizo sobre df['text'], entonces podemos usar el df original.
    if 'helpful' in cluster_df.columns:
        print("\nEstadísticas descriptivas de Votos Útiles ('helpful'):")
        print(cluster_df['helpful'].describe())
    else:
        print("\nLa columna 'helpful' no está disponible en el DataFrame del cluster.")


    # Análisis de la variable 'review_length'
    if 'review_length' in cluster_df.columns:
        print("\nEstadísticas descriptivas de la Longitud de la Reseña:")
        print(cluster_df['review_length'].describe())
    else:
         print("\nLa columna 'review_length' no está disponible en el DataFrame del cluster.")


    # Análisis de las palabras más frecuentes en el texto del cluster
    # Puedes tomar una muestra si el cluster es muy grande
    sample_text = ' '.join(cluster_df['text'].sample(min(1000, num_reviews), replace=True).tolist())

    # Usar el mismo vectorizador TF-IDF (o uno nuevo si quieres) para analizar el texto del cluster
    # Para simple frecuencia, puedes usar CountVectorizer o analizar la cadena directamente
    from collections import Counter
    import re

    words = re.findall(r'\b\w+\b', sample_text.lower()) # Extrae palabras, convierte a minúsculas
    stop_words = set(['el', 'la', 'los', 'las', 'un', 'una', 'unos', 'unas', 'de', 'en', 'y', 'a', 'que', 'es', 'con', 'para', 'del', 'al', 'se', 'por', 'su', 'sus', 'como', 'mas', 'pero', 'este', 'esta', 'si', 'no', 'cuando', 'donde', 'porque', 'muy', 'mi', 'mis', 'tu', 'tus', 'el', 'ella', 'nosotros', 'vosotros', 'ellos', 'las', 'los']) # Ejemplo de stop words en español

    # Filtrar stop words
    filtered_words = [word for word in words if word not in stop_words and len(word) > 1]

    word_counts = Counter(filtered_words)
    print("\nPalabras más frecuentes (Top 10):")
    print(word_counts.most_common(10))

    print("-" * (len(f"--- Cluster {cluster_id} ---"))) # Separador para el siguiente cluster

### 7. Insight & Recomendaciones (15 pts)

*Escribe aquí tu análisis de cómo se relacionan los errores del modelo con los clusters y propone 1‑2 acciones de negocio basadas en tus hallazgos.*

<!-- TODO -->

**Análisis de errores del modelo y su relación con los grupos (clusters):**
El modelo de regresión logística que usamos para clasificar reseñas como positivas (4 o más estrellas) o negativas muestra buenos resultados generales, con un 92% de precisión. Sin embargo, si miramos más de cerca la matriz de confusión y el informe de desempeño, vemos que aún hay puntos a mejorar, sobre todo al identificar reseñas negativas, que son menos comunes.

- Falsos negativos (FN): El modelo clasificó 90 reseñas negativas como si fueran positivas. Esto hace que su capacidad para detectar reseñas negativas (recall) sea baja, con solo un 78% de aciertos en esa clase. Esto es preocupante, ya que se pueden perder oportunidades de detectar fallas en el producto o servicio.

- Falsos positivos (FP): El modelo también clasificó 31 reseñas positivas como si fueran negativas. Aunque no son muchas, pueden llevar a decisiones innecesarias o malinterpretaciones sobre la satisfacción del cliente.

- Desbalance entre clases: Solo el 20% de las reseñas son negativas, lo que hace más difícil para el modelo aprender a identificarlas correctamente. Aunque usamos una técnica para balancear el modelo (class_weight='balanced'), el problema persiste, algo común cuando hay mucha más cantidad de una clase que de otra.

- Relación con los grupos (clusters):
Al analizar los grupos creados con clustering, vimos que algunos tienen más reseñas negativas que otros. Por ejemplo, el Cluster 1 tiene casi un 23% de reseñas negativas, mientras que otros, como el Cluster 0 y el 2, son casi totalmente positivos. Es probable que muchos de los errores del modelo (como los falsos negativos) estén dentro de los grupos con opiniones mixtas, donde el texto es más difícil de interpretar. En cambio, los clusters con reseñas mayoritariamente positivas ayudan a que el modelo tenga un buen rendimiento general.

**Recomendaciones para el negocio:**
1. Analizar errores en clusters específicos: Se deberían revisar las reseñas que el modelo clasificó mal (especialmente las negativas que trató como positivas), enfocándose en los grupos con más reseñas negativas, como el Cluster 1. Esto puede hacerse de forma manual o automática para ver si hay palabras, tonos (como sarcasmo) o temas que el modelo no está entendiendo. Esta información serviría para mejorar el procesamiento del texto o incluso probar modelos más avanzados.

2. Usar los clusters como apoyo: Los clusters pueden ayudar a entender mejor las reseñas antes de clasificarlas. Por ejemplo, se puede dar prioridad a los clusters con más reseñas negativas para análisis más detallados. También se podrían crear modelos especializados para cada cluster. Además, el análisis de las palabras comunes en cada grupo puede dar pistas sobre lo que más gusta o molesta a los clientes, lo cual puede usarse para tomar decisiones en marketing, producto o servicio al cliente.


## ✅ 8. Checklist final
- [ ] Notebook corre sin errores
- [ ] Respuestas teóricas completadas
- [ ] Comentarios claros y semillas fijas
- [ ] 5 slides creadas y exportadas a PDF
